In [87]:
import imp
imp.reload(main)
imp.reload(regression_models)

<module 'run_model.regression_models' from 'C:\\Users\\Owner\\GIT\\DataSci\\run_model\\regression_models.py'>

In [88]:
from aggregate import aggregate
from aggregate import capm_params
from feature_handler import feature_handler
from run_model import regression_models
import pandas as pd
import numpy as np
import main
import random
import matplotlib.pyplot as plt
import seaborn as sb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
sb.set()

### Step 0: Notebook intialization
seed: 
>We ran the notebook using seed = 10

run_speed:
>3 - fastest (immediate)\
    2 - avrage\
    1 - slowest

In [89]:
seed = 10
random.seed(seed)
np.random.seed(seed)
run_speed = 3

### Step 1: data pre-process steps and initial feature extraction
This step include very intial data pre-processing:
1. Drop NaNs
2. Take care of data types
3. Add dividend change and dividend change direction
4. Add the abnormal return feature (which is the value we will try to predict and explain)


In [90]:
df = pd.read_csv("aggregated_data.csv")

rows_before = df.shape[0]
df.dropna(inplace=True)
rows_after = df.shape[0]
print("Dropped {} rows with NaN values".format(rows_before-rows_after))

# dividend_amount to integer
df = feature_handler.create_div_amount_num(df)
df.drop(df[df["div_amount_num"] == 0].index, inplace=True)
df.drop("dividend_amount", axis = 1, inplace = True)

# Add dividend direction and dividend change
df = feature_handler.gen_div_direction_and_change(df)

# Add abnormal return data
df = feature_handler.create_abnormal_return(df)
df.reset_index()
if run_speed <= 1: df.to_csv("data_with_ar.csv", index=False)

Dropped 86 rows with NaN values
################# Creating div_amount_num #################
Found 77 samples were dividend_amount did not start with $ - drop
Found 4 divdends that were eqaul to 0 - drop
>>> Finished. Total dropped rows due to invalid dividend info: 81
################# Creating div_direction and div_change ###
>>> Finished
################# Creating abnormal return related features ###
	feature type = expected_t		11 features
	feature type = ar_t		11 features
	feature type = aar_t		6 features
	feature type = aar_t%		6 features
>>> Finished. Created 34 features


In [91]:
display(df.head(3))

,company_name,symbol,industry,sector,dividend_date,alpha,beta,price_t-5,vol_t-5,sp_price_t-5,...,aar_2,aar_3,aar_4,aar_5,aar_0%,aar_1%,aar_2%,aar_3%,aar_4%,aar_5%
0,"American Airlines Group, Inc.",AAL,Air Freight/Delivery Services,Transportation,2019-10-22,-33.847291,0.020567,27.930000,6709600.0,2997.949951,...,1.724005,2.212617,2.456924,2.497174,0.017549,0.017393,0.024743,0.033998,0.039134,0.040636
1,"American Airlines Group, Inc.",AAL,Air Freight/Delivery Services,Transportation,2019-07-24,-29.842482,0.021049,33.080002,4249000.0,2976.610107,...,-1.616090,-1.858569,-1.979809,-2.131966,0.025877,-0.005287,-0.019312,-0.023825,-0.026337,-0.029026
2,"American Airlines Group, Inc.",AAL,Air Freight/Delivery Services,Transportation,2019-04-24,-80.607557,0.039610,33.480000,3884200.0,2907.969971,...,-4.369771,-4.205517,-4.123390,-3.957069,-0.038206,-0.045773,-0.049474,-0.051065,-0.051951,-0.050990


### Step 2: Data insights and visualization including drop outliers

In [92]:
_ = main.print_basic_stats(df)

Data Stats:
	#Samples: 39746
	#Features: 98
	#Samples with NaNs: 0


In [93]:
df = main.remove_outliers(df, range_min = -100 , range_max = 100)

Removed 36 outliers


# Step 3: Base line model
Now we will try to fit a linear regression model to our data, to use it as a baseline.\
We will consider two setups:
1. Follow the research and look only at the samples where $div\_change \neq 0$, our goal it to try replicate the result found in earlier research
2. Use all of our sampels

Remeber we are trying to predict "aar5", hence we will not use any related features, that leaves us only with

In [97]:
baseline_models_data = regression_models.generate_bl_model_data(df, print_ = False)

In [98]:
#Regression model 1
_ = regression_models.run_linear_reg_research(baseline_models_data)

                            OLS Regression Results                            
Dep. Variable:                  aar_5   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     14.84
Date:                Mon, 24 Feb 2020   Prob (F-statistic):           5.23e-41
Time:                        13:53:39   Log-Likelihood:                -35794.
No. Observations:               12784   AIC:                         7.162e+04
Df Residuals:                   12767   BIC:                         7.175e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
index       

In [99]:
#Regression model 2
_ = regression_models.run_linear_reg_baseline(baseline_models_data)

                            OLS Regression Results                            
Dep. Variable:                  aar_5   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     10.81
Date:                Mon, 24 Feb 2020   Prob (F-statistic):           2.50e-28
Time:                        13:53:46   Log-Likelihood:            -1.1092e+05
No. Observations:               39710   AIC:                         2.219e+05
Df Residuals:                   39693   BIC:                         2.220e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
index       

TODO: Discuss the results

### Step 4: Analysis of error for the base line model+ conclusions (years 2008-2009)
TODO: visualize and discuss

### Step 5: Sensitivity analysis (size of window)

### Step 6: Run new model (Regression) (naive + feature handler)

### Step 7: Run different Regression models and compare

### Step 8: Discrete Models

### Step 9: Cross Validation, results and conclusions